# Tests utilisation de la base de donnée

In [71]:
import numpy as np
import pandas as pd

In [72]:
def convert_attributes_into_pandas(filename):
    return pd.read_csv(filename)

In [73]:
df_attributes = convert_attributes_into_pandas("attributes_data.csv")
df_attributes

,ID,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
202595,202596.jpg,-1,-1,-1,-1,-1,1,1,-1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
202596,202597.jpg,-1,-1,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
202597,202598.jpg,-1,1,1,-1,-1,-1,1,-1,1,...,-1,1,-1,1,1,-1,1,-1,-1,1


In [74]:
indexes = df_attributes.columns
indexes

Index(['ID', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

## Réduction matrice

In [76]:
#On souhaite pouvoir réduire la matrice selon certaines colonnes, dont les attributs correspondants seront fixés
#et ne pourront donc pas changer pendant l'algorithme génétique.
#Pour cela on va récupérer une liste d'attributs avec certaines valeurs et utiliser cette liste pour réduire la matrice

fixed_attributes = {"Attractive" : 1 ,"Eyeglasses" : 1 , "Gray_Hair" : -1 , "Brown_Hair" : -1 , "Black_Hair" : 1 ,"Blond_Hair" : -1 , "Pale_Skin":1}

In [79]:
def matrix_reduction(df_attributes,fixed_attributes):
    attributes = list(fixed_attributes.keys())
    values = list(fixed_attributes.values())
    for i in range(len(attributes)):
        df_attributes.drop(df_attributes.index[df_attributes[attributes[i]]!= values[i]],inplace=True)
    return df_attributes


In [80]:
matrix_reduction(df_attributes,fixed_attributes)

,ID,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
391,000392.jpg,-1,-1,1,-1,-1,-1,1,-1,1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
16738,016739.jpg,-1,-1,1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
24434,024435.jpg,-1,-1,1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
25603,025604.jpg,-1,-1,1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
31578,031579.jpg,-1,-1,1,-1,-1,-1,1,-1,1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
43012,043013.jpg,-1,-1,1,-1,-1,-1,1,-1,1,...,-1,-1,-1,1,-1,-1,1,-1,-1,1
46698,046699.jpg,-1,-1,1,-1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,-1,-1,1
58089,058090.jpg,-1,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,-1,-1,-1,1,-1,1
68812,068813.jpg,-1,-1,1,-1,-1,-1,1,-1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
70470,070471.jpg,-1,-1,1,-1,-1,-1,-1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
